In [ ]:
import pandas as pd
import plotly_express as px

In [ ]:
#reading the csv file
df = pd.read_csv("flood_control_data.csv", sep=",")

In [ ]:
df["ABC"]

In [ ]:
#retrieving the min and max of the infra year
df

In [ ]:
#retrieve all region
df[df["Region"] == "Region I"]["Province"].unique()

In [ ]:
#get the total alocated funds default
df["StartDate"] = pd.to_datetime(df["StartDate"], format="%d/%m/%Y")

In [ ]:
#get the timeseries projects by month of the year
df["monthyear"] = df["StartDate"].apply(lambda araw: str(araw.month) + str(araw.year))

In [ ]:
df.groupby("monthyear")

In [ ]:
df["ProjectID"].nunique()

In [ ]:
df[df["ProjectID"].duplicated()]

In [ ]:
df[df["ProjectID"] == "P00421352LZ"].reset_index(drop=True)

In [ ]:
import plotly_express as px

In [ ]:
#generating the number of projects over time
dat = df.groupby("StartDate").agg({
    "ProjectID": "count",
    "ABC": "sum"
}).reset_index().rename(columns={"ProjectID": "Total Projects", "ABC":"Total Spent"})


In [ ]:
fig = px.line(dat, x="StartDate", y="Total Projects")
fig.update_layout(
    plot_bgcolor="white",
    paper_bgcolor="white"
)
fig.show()

In [ ]:
#how many projects started per quarter
df["quarter"] = df["StartDate"].dt.quarter

In [ ]:
qua = df.groupby("quarter")["ProjectID"].nunique().reset_index(name="total projects")

In [ ]:
fig2 = px.pie(qua, names="quarter", values="total projects")
fig2.show()

In [2]:
import dash
from dash import html
import dash_cytoscape as cyto
import pandas as pd

# Sample of your dataset
data = {
    "ProjectID": ["P00824759LZ", "P00824758LZ", "P00824750LZ"],
    "ProjectDescription": [
        "Babuyan River Flood Control Downstream",
        "Langogan River Flood Control",
        "Babuyan River Flood Control Upstream"
    ],
    "Contractor": ["AZARRAGA CONSTRUCTION", "AZARRAGA CONSTRUCTION", "F.F. GALANG CONSTRUCTION"]
}
df = pd.DataFrame(data)

# Build Cytoscape elements
elements = []

# Add contractor nodes
for contractor in df["Contractor"].unique():
    elements.append({
        "data": {"id": contractor, "label": contractor},
        "classes": "contractor"
    })

# Add project nodes + edges to contractor
for _, row in df.iterrows():
    elements.append({
        "data": {"id": row["ProjectID"], "label": row["ProjectDescription"]},
        "classes": "project"
    })
    elements.append({
        "data": {"source": row["Contractor"], "target": row["ProjectID"]}
    })

# Dash app
app = dash.Dash(__name__)
app.layout = html.Div([
    cyto.Cytoscape(
        id="network",
        elements=elements,
        style={"width": "100%", "height": "600px"},
        layout={"name": "cose"},  # force-directed layout
        stylesheet=[
            {
                "selector": ".contractor",
                "style": {"background-color": "red", "label": "data(label)", "shape": "rectangle"}
            },
            {
                "selector": ".project",
                "style": {"background-color": "blue", "label": "data(label)"}
            }
        ]
    )
])

if __name__ == "__main__":
    app.run_server(debug=True)


OSError: Address 'http://127.0.0.1:8050' already in use.
    Try passing a different port to run.